In [ ]:
from google.colab import drive
drive.mount('/content/drive')#Mount the drive

Mounted at /content/drive


In [ ]:
! pip install tensorflow

Load the Dataset

In [ ]:
import numpy as np
import plotly.graph_objects as go
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.regularizers import l2
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Activation
# Load data
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [ ]:
# Normalize the features
scaler = StandardScaler()
x_train_norm = scaler.fit_transform(x_train)
x_test_norm = scaler.transform(x_test)

Build the Neural Network Model

In [ ]:
# Define Neural Network Model 1
model_1 = Sequential([
    Dense(13, input_shape=(13,), use_bias=False),  # No activation function in the input layer
    Dense(13, activation='relu'),
    Dense(13, activation='relu'),
    Dense(13, activation='relu'),
    Dense(1, activation=None)  # Output layer
])

In [ ]:
# Compile the model
model_1.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
# Custom R2 Callback
class R2Callback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data
        self.r2_scores = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.validation_data[0])
        r2 = r2_score(self.validation_data[1], y_pred)
        self.r2_scores.append(r2)
        print(f"Epoch {epoch+1}: val_r2 = {r2:.4f}")

In [ ]:
# Initialize the R2 Callback
r2_callback = R2Callback(validation_data=(x_test_norm, y_test))

In [ ]:
# Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=10,
    verbose=1,
    restore_best_weights=True
)

In [ ]:
# Train the model with normalized data and callbacks
history_1 = model_1.fit(
    x_train_norm, y_train,
    epochs=100,
    verbose=1,
    validation_data=(x_test_norm, y_test),
    callbacks=[early_stopping, r2_callback]
)

Epoch 1/100
4/4 [==============================] - 0s 3ms/step
Epoch 1: val_r2 = -6.1477
13/13 [==============================] - 2s 51ms/step - loss: 572.6603 - val_loss: 595.0014
Epoch 2/100
4/4 [==============================] - 0s 3ms/step
Epoch 2: val_r2 = -5.9279
13/13 [==============================] - 0s 15ms/step - loss: 559.8852 - val_loss: 576.7071
Epoch 3/100
4/4 [==============================] - 0s 2ms/step
Epoch 3: val_r2 = -5.5976
13/13 [==============================] - 0s 17ms/step - loss: 537.9920 - val_loss: 549.2079
Epoch 4/100
4/4 [==============================] - 0s 3ms/step
Epoch 4: val_r2 = -5.1143
13/13 [==============================] - 0s 16ms/step - loss: 506.9301 - val_loss: 508.9798
Epoch 5/100
4/4 [==============================] - 0s 3ms/step
Epoch 5: val_r2 = -4.4273
13/13 [==============================] - 0s 16ms/step - loss: 463.0247 - val_loss: 451.7858
Epoch 6/100
4/4 [==============================] - 0s 3ms/step
Epoch 6: val_r2 = -3.4726
13/13 

In [ ]:
# Model summary
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 13)                169       
                                                                 
 dense_1 (Dense)             (None, 13)                182       
                                                                 
 dense_2 (Dense)             (None, 13)                182       
                                                                 
 dense_3 (Dense)             (None, 13)                182       
                                                                 
 dense_4 (Dense)             (None, 1)                 14        
                                                                 
Total params: 729 (2.85 KB)
Trainable params: 729 (2.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Plot Loss Over Epochs
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, len(history_1.history['loss']) + 1)), y=history_1.history['loss'], mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(history_1.history['val_loss']) + 1)), y=history_1.history['val_loss'], mode='lines', name='Validation Loss'))
fig.update_layout(title='Loss Over Epochs', xaxis_title='Epochs', yaxis_title='Loss')
fig.show()

the model will stop training early if the validation loss does not improve by at least 0.001 for 10 consecutive epochs, and it will restore the weights from the epoch that had the best validation loss.

In [ ]:
# Plot R2 Over Epochs
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, len(r2_callback.r2_scores) + 1)), y=r2_callback.r2_scores, mode='lines', name='Validation R2'))
fig.update_layout(title='R2 Score Over Epochs', xaxis_title='Epochs', yaxis_title='R2 Score')
fig.show()

In [ ]:
# Predict and evaluate using normalized test data
y_pred = model_1.predict(x_test_norm)
final_r2 = r2_score(y_test, y_pred)
print(f'Final R2 Score: {final_r2:.4f}')

4/4 [==============================] - 0s 3ms/step
Final R2 Score: 0.6518


In [ ]:
# Plot Predictions vs Actual - Statistical Line Fit
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=y_pred.flatten(), mode='markers', name='Predictions vs Actual'))
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='Ideal Fit'))
fig.update_layout(title='Predictions vs Actual Values - Statistical Line Fit', xaxis_title='Actual Values', yaxis_title='Predicted Values')
fig.show()

In [ ]:
# Plot Prediction vs Actual - Connect Line
fig = go.Figure()
for i in range(len(y_test)):
    fig.add_trace(go.Scatter(x=[y_test[i], y_test[i]], y=[y_test[i], y_pred.flatten()[i]], mode='lines+markers', name=f'Point {i}'))
fig.update_layout(title='Prediction vs Actual Values - Connect Line', xaxis_title='Actual Values', yaxis_title='Predicted Values')
fig.show()

In [ ]:
# Calculate the final R2 score for reporting
final_r2 = r2_score(y_test, y_pred)
print(f'Final R2 Score: {final_r2:.4f}')

Final R2 Score: 0.6518


In [ ]:
# Normalize the features
scaler = StandardScaler()
x_train_norm = scaler.fit_transform(x_train)
x_test_norm = scaler.transform(x_test)

In [ ]:
# Define the model_2 with RELU activation in the input layer
model_2 = Sequential([
    Dense(13, input_shape=(13,)),
    Activation('relu'),
    Dense(13, activation='relu'),
    Dense(13, activation='relu'),
    Dense(13, activation='relu'),
    Dense(1)
])

In [ ]:
# Compile the model_2
model_2.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')

In [ ]:
# R2 Callback
class R2Callback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data
        self.r2_scores = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.validation_data[0])
        r2 = r2_score(self.validation_data[1], y_pred)
        self.r2_scores.append(r2)

In [ ]:
# Initialize the R2 Callback
r2_callback = R2Callback(validation_data=(x_test_norm, y_test))

In [ ]:
# Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=10,
    verbose=1,
    restore_best_weights=True
)

In [ ]:
# Train the model_2 with normalized data and callbacks
history_2 = model_2.fit(
    x_train_norm, y_train,
    epochs=100,
    verbose=1,
    validation_data=(x_test_norm, y_test),
    callbacks=[early_stopping, r2_callback]
)

Epoch 1/100
13/13 [==============================] - 2s 26ms/step - loss: 583.3657 - val_loss: 613.3104
Epoch 2/100
13/13 [==============================] - 0s 11ms/step - loss: 581.6091 - val_loss: 611.5267
Epoch 3/100
13/13 [==============================] - 0s 11ms/step - loss: 579.9048 - val_loss: 609.7017
Epoch 4/100
13/13 [==============================] - 0s 14ms/step - loss: 578.1384 - val_loss: 607.8809
Epoch 5/100
13/13 [==============================] - 0s 18ms/step - loss: 576.3927 - val_loss: 606.0235
Epoch 6/100
13/13 [==============================] - 0s 18ms/step - loss: 574.6396 - val_loss: 604.0974
Epoch 7/100
13/13 [==============================] - 0s 23ms/step - loss: 572.8162 - val_loss: 602.0977
Epoch 8/100
13/13 [==============================] - 0s 20ms/step - loss: 570.9040 - val_loss: 600.0880
Epoch 9/100
13/13 [==============================] - 0s 19ms/step - loss: 568.9892 - val_loss: 597.9975
Epoch 10/100
13/13 [==============================] - 0s 18ms/st

In [ ]:
# Model summary
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 13)                182       
                                                                 
 activation_1 (Activation)   (None, 13)                0         
                                                                 
 dense_12 (Dense)            (None, 13)                182       
                                                                 
 dense_13 (Dense)            (None, 13)                182       
                                                                 
 dense_14 (Dense)            (None, 13)                182       
                                                                 
 dense_15 (Dense)            (None, 1)                 14        
                                                                 
Total params: 742 (2.90 KB)
Trainable params: 742 (2.9

In [ ]:
# Loss Over Epochs
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, len(history_2.history['loss']) + 1)), y=history_2.history['loss'], mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(history_2.history['val_loss']) + 1)), y=history_2.history['val_loss'], mode='lines', name='Validation Loss'))
fig.update_layout(title='Loss Over Epochs for Model 2', xaxis_title='Epochs', yaxis_title='Loss')
fig.show()

In [ ]:
# R2 Over Epochs
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, len(r2_callback.r2_scores) + 1)), y=r2_callback.r2_scores, mode='lines', name='Validation R2'))
fig.update_layout(title='R2 Score Over Epochs for Model 2', xaxis_title='Epochs', yaxis_title='R2 Score')
fig.show()

In [ ]:
# Predict and evaluate using normalized test data
y_pred = model_2.predict(x_test_norm)

4/4 [==============================] - 0s 7ms/step


In [ ]:
# Regression report
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
final_r2 = r2_score(y_test, y_pred)
print(f'Regression Report for Model 2:\nMSE: {mse:.4f}\nMAE: {mae:.4f}\nR2 Score: {final_r2:.4f}')

Regression Report for Model 2:
MSE: 60.1997
MAE: 6.4338
R2 Score: 0.2768


In [ ]:
# Predictions vs Actual - Statistical Line Fit
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=y_pred.flatten(), mode='markers', name='Predictions vs Actual'))
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='Ideal Fit'))
fig.update_layout(title='Predictions vs Actual Values - Statistical Line Fit for Model 2', xaxis_title='Actual Values', yaxis_title='Predicted Values')
fig.show()

In [ ]:
# Prediction vs Actual Values - Connect Line
fig = go.Figure()
for i in range(len(y_test)):
    fig.add_trace(go.Scatter(x=[y_test[i], y_test[i]], y=[y_test[i], y_pred.flatten()[i]], mode='lines+markers', name=f'Point {i}'))
fig.update_layout(title='Prediction vs Actual Values - Connect Line for Model 2', xaxis_title='Actual Values', yaxis_title='Predicted Values')
fig.show()

In [ ]:
# Calculate the final R2 score for reporting
final_r2_2 = r2_score(y_test, y_pred)
print(f'Final R2 Score for Model 2: {final_r2_2:.4f}')

Final R2 Score for Model 2: 0.2768


In [ ]:
# Define Neural Network Model 3 with dropout layers for regularization
model_3 = Sequential([
    Dense(64, input_shape=(13,), activation='relu', kernel_regularizer='l2'),  # More nodes in the input layer
    Dropout(0.2),  # Dropout layer after the input layer
    Dense(64, activation='relu', kernel_regularizer='l2'),
    Dropout(0.2),  # Adding dropout to subsequent layers
    Dense(64, activation='relu', kernel_regularizer='l2'),
    Dropout(0.2),
    Dense(64, activation='relu', kernel_regularizer='l2'),
    Dropout(0.2),
    Dense(1, activation=None)  # Output layer
])

In [ ]:
# Compile model with RMSProp optimizer and a learning rate of 0.0001
model_3.compile(optimizer=RMSprop(learning_rate=0.0001), loss='mse')

In [ ]:
# Custom R2 Callback to record R2 score at the end of each epoch
class R2Callback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data
        self.r2_scores = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.validation_data[0])
        r2 = r2_score(self.validation_data[1], y_pred)
        self.r2_scores.append(r2)

In [ ]:
r2_callback = R2Callback(validation_data=(x_test_norm, y_test))

In [ ]:
# Train the model with normalized data for 100 epochs
history_3 = model_3.fit(
    x_train_norm, y_train,
    epochs=100,
    verbose=1,
    validation_data=(x_test_norm, y_test),
    callbacks=[r2_callback]
)

Epoch 1/100
13/13 [==============================] - 1s 29ms/step - loss: 584.8486 - val_loss: 609.4305
Epoch 2/100
13/13 [==============================] - 0s 13ms/step - loss: 577.9736 - val_loss: 603.3601
Epoch 3/100
13/13 [==============================] - 0s 13ms/step - loss: 573.0256 - val_loss: 597.1443
Epoch 4/100
13/13 [==============================] - 0s 11ms/step - loss: 564.6174 - val_loss: 590.2952
Epoch 5/100
13/13 [==============================] - 0s 16ms/step - loss: 557.8262 - val_loss: 582.8761
Epoch 6/100
13/13 [==============================] - 0s 17ms/step - loss: 549.2097 - val_loss: 574.4561
Epoch 7/100
13/13 [==============================] - 0s 17ms/step - loss: 540.6478 - val_loss: 565.2325
Epoch 8/100
13/13 [==============================] - 0s 18ms/step - loss: 530.8016 - val_loss: 555.1260
Epoch 9/100
13/13 [==============================] - 0s 16ms/step - loss: 520.0854 - val_loss: 544.3000
Epoch 10/100
13/13 [==============================] - 0s 16ms/st

In [ ]:
# Model summary
model_3.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_89 (Dense)            (None, 64)                896       
                                                                 
 dropout_60 (Dropout)        (None, 64)                0         
                                                                 
 dense_90 (Dense)            (None, 64)                4160      
                                                                 
 dropout_61 (Dropout)        (None, 64)                0         
                                                                 
 dense_91 (Dense)            (None, 64)                4160      
                                                                 
 dropout_62 (Dropout)        (None, 64)                0         
                                                                 
 dense_92 (Dense)            (None, 64)              

In [ ]:
# Loss Over Epochs using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, 101)), y=history_3.history['loss'], mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=list(range(1, 101)), y=history_3.history['val_loss'], mode='lines', name='Validation Loss'))
fig.update_layout(title='Loss Over Epochs for Model 3', xaxis_title='Epochs', yaxis_title='Loss')
fig.show()

In [ ]:
# R2 Over Epochs using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, 101)), y=r2_callback.r2_scores, mode='lines', name='Validation R2'))
fig.update_layout(title='R2 Score Over Epochs for Model 3', xaxis_title='Epochs', yaxis_title='R2 Score')
fig.show()

In [ ]:
# Predictions vs Actual Values - Statistical Line Fit
y_pred_3 = model_3.predict(x_test_norm)
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=y_pred_3.flatten(), mode='markers', name='Predictions vs Actual'))
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='Ideal Fit'))
fig.update_layout(title='Predictions vs Actual Values - Statistical Line Fit for Model 3', xaxis_title='Actual Values', yaxis_title='Predicted Values')
fig.show()

4/4 [==============================] - 0s 3ms/step


In [ ]:
# Prediction vs Actual Values - Connect Line
fig = go.Figure()
for i in range(len(y_test)):
    fig.add_trace(go.Scatter(x=[y_test[i], y_test[i]], y=[y_test[i], y_pred_3.flatten()[i]], mode='lines+markers', name=f'Point {i}'))
fig.update_layout(title='Prediction vs Actual Values - Connect Line for Model 3', xaxis_title='Actual Values', yaxis_title='Predicted Values')
fig.show()

In [ ]:
# Calculate the final R2 score for reporting
final_r2_3 = r2_score(y_test, y_pred_3)
print(f'Final R2 Score for Model 3: {final_r2_3:.4f}')

Final R2 Score for Model 3: 0.6870


In [ ]:
# Define Neural Network Model 4 with additional dense layers
model_4 = Sequential([
    Dense(64, input_shape=(13,), activation='relu'),
    Dropout(0.1),  # Slightly lower dropout initially
    Dense(128, activation='relu'),  # Increasing the number of neurons
    Dropout(0.2),  # Higher dropout for a larger layer
    Dense(128, activation='relu'),
    Dropout(0.3),  # Even higher dropout to prevent overfitting in deeper layers
    Dense(64, activation='relu'),
    Dropout(0.2),  # Reducing dropout as we approach the output layer
    Dense(32, activation='relu'),  # Adding an additional layer
    Dropout(0.1),
    Dense(1, activation=None)  # Output layer
])


In [ ]:

# Compile model with ADAM optimizer and a learning rate of 0.0001
model_4.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')

In [ ]:
# Custom R2 Callback to record R2 score at the end of each epoch
class R2Callback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data
        self.r2_scores = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.validation_data[0])
        r2 = r2_score(self.validation_data[1], y_pred)
        self.r2_scores.append(r2)

r2_callback = R2Callback(validation_data=(x_test_norm, y_test))

In [ ]:
# Early Stopping to stop training when the validation loss has stopped improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=20,  # Increased patience
    verbose=1,
    restore_best_weights=True
)

In [ ]:
# Train the model with normalized data
history_4 = model_4.fit(
    x_train_norm, y_train,
    epochs=100,  # Maximum number of epochs
    verbose=1,
    validation_data=(x_test_norm, y_test),
    callbacks=[early_stopping, r2_callback]
)


Epoch 1/100
13/13 [==============================] - 2s 46ms/step - loss: 590.3491 - val_loss: 614.6915
Epoch 2/100
13/13 [==============================] - 0s 23ms/step - loss: 585.8102 - val_loss: 610.3014
Epoch 3/100
13/13 [==============================] - 0s 22ms/step - loss: 580.4816 - val_loss: 605.8134
Epoch 4/100
13/13 [==============================] - 0s 23ms/step - loss: 575.4709 - val_loss: 600.2807
Epoch 5/100
13/13 [==============================] - 0s 13ms/step - loss: 569.6149 - val_loss: 593.0862
Epoch 6/100
13/13 [==============================] - 0s 13ms/step - loss: 562.2584 - val_loss: 583.4523
Epoch 7/100
13/13 [==============================] - 0s 12ms/step - loss: 550.7012 - val_loss: 570.2896
Epoch 8/100
13/13 [==============================] - 0s 13ms/step - loss: 536.1140 - val_loss: 552.4340
Epoch 9/100
13/13 [==============================] - 0s 16ms/step - loss: 516.8381 - val_loss: 528.4318
Epoch 10/100
13/13 [==============================] - 0s 13ms/st

In [ ]:
# Model summary
model_4.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_94 (Dense)            (None, 64)                896       
                                                                 
 dropout_64 (Dropout)        (None, 64)                0         
                                                                 
 dense_95 (Dense)            (None, 128)               8320      
                                                                 
 dropout_65 (Dropout)        (None, 128)               0         
                                                                 
 dense_96 (Dense)            (None, 128)               16512     
                                                                 
 dropout_66 (Dropout)        (None, 128)               0         
                                                                 
 dense_97 (Dense)            (None, 64)              

In [ ]:
# Plot Loss Over Epochs using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, len(history_4.history['loss']) + 1)), y=history_4.history['loss'], mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(history_4.history['val_loss']) + 1)), y=history_4.history['val_loss'], mode='lines', name='Validation Loss'))
fig.update_layout(title='Loss Over Epochs for Model 4', xaxis_title='Epochs', yaxis_title='Loss')
fig.show()

In [ ]:
# R2 Over Epochs using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, len(r2_callback.r2_scores) + 1)), y=r2_callback.r2_scores, mode='lines', name='Validation R2'))
fig.update_layout(title='R2 Score Over Epochs for Model 4', xaxis_title='Epochs', yaxis_title='R2 Score')
fig.show()

In [ ]:
# Predictions vs Actual Values - Statistical Line Fit
y_pred_4 = model_4.predict(x_test_norm)
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=y_pred_4.flatten(), mode='markers', name='Predictions vs Actual'))
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='Ideal Fit'))
fig.update_layout(title='Predictions vs Actual Values - Statistical Line Fit for Model 4', xaxis_title='Actual Values', yaxis_title='Predicted Values')
fig.show()

4/4 [==============================] - 0s 2ms/step


In [ ]:
# Prediction vs Actual Values - Connect Line
fig = go.Figure()
for i in range(len(y_test)):
    fig.add_trace(go.Scatter(x=[y_test[i], y_test[i]], y=[y_test[i], y_pred_4.flatten()[i]], mode='lines+markers', name=f'Point {i}'))
fig.update_layout(title='Prediction vs Actual Values - Connect Line for Model 4', xaxis_title='Actual Values', yaxis_title='Predicted Values')
fig.show()

In [ ]:
# Calculate the final R2 score for reporting
final_r2_4 = r2_score(y_test, y_pred_4)
print(f'Final R2 Score for Model 4: {final_r2_4:.4f}')

Final R2 Score for Model 4: 0.7201


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# First, we will need predictions from each model on the test set
# Assuming x_test_norm is your normalized test set features and y_test are the actual target values
y_pred_1 = model_1.predict(x_test_norm)
y_pred_2 = model_1.predict(x_test_norm)
y_pred_3 = model_3.predict(x_test_norm)
y_pred_4 = model_4.predict(x_test_norm)

# Define a function to print regression report for a model
def print_regression_report(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f'Regression Report for {model_name}:')
    print(f'Mean Squared Error (MSE): {mse:.4f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
    print(f'Mean Absolute Error (MAE): {mae:.4f}')
    print(f'R-squared (R²): {r2:.4f}')
    print('-----------------------------------\n')

# Print out the regression reports for each model
print_regression_report(y_test, y_pred_1, 'Model 1')
print_regression_report(y_test, y_pred_1, 'Model 2')
print_regression_report(y_test, y_pred_3, 'Model 3')
print_regression_report(y_test, y_pred_4, 'Model 4')


4/4 [==============================] - 0s 3ms/step
Regression Report for Model 1:
Mean Squared Error (MSE): 28.9867
Root Mean Squared Error (RMSE): 5.3839
Mean Absolute Error (MAE): 4.1924
R-squared (R²): 0.6518
-----------------------------------

Regression Report for Model 2:
Mean Squared Error (MSE): 28.9867
Root Mean Squared Error (RMSE): 5.3839
Mean Absolute Error (MAE): 4.1924
R-squared (R²): 0.6518
-----------------------------------

Regression Report for Model 3:
Mean Squared Error (MSE): 26.0580
Root Mean Squared Error (RMSE): 5.1047
Mean Absolute Error (MAE): 3.8340
R-squared (R²): 0.6870
-----------------------------------

Regression Report for Model 4:
Mean Squared Error (MSE): 23.2986
Root Mean Squared Error (RMSE): 4.8269
Mean Absolute Error (MAE): 3.5312
R-squared (R²): 0.7201
-----------------------------------



In [ ]:
model_1.save('model_1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [ ]:
model_2.save('model_2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [ ]:
model_3.save('model_3.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [ ]:
model_4.save('model_4.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [ ]:
model_1.save('model_1', save_format='tf')
model_2.save('model_2', save_format='tf')
model_3.save('model_3', save_format='tf')
model_4.save('model_4', save_format='tf')


In [ ]:
# Function to serialize model's weights and configuration
def serialize_model(model):
    return {
        'weights': model.get_weights(),
        'config': model.to_json()  # Using to_json for the model's configuration
    }

# Serialize each model
serialized_model_1 = serialize_model(model_1)
serialized_model_2 = serialize_model(model_2)
serialized_model_3 = serialize_model(model_3)
serialized_model_4 = serialize_model(model_4)


In [ ]:
import pickle

# Combine serialized data into a dictionary
all_models_serialized = {
    'model_1': serialized_model_1,
    'model_2': serialized_model_2,
    'model_3': serialized_model_3,
    'model_4': serialized_model_4
}

# Pickle the dictionary containing all models
with open('all_models.pkl', 'wb') as file:
    pickle.dump(all_models_serialized, file)
